In [1]:
import mxnet as mx
from mxnet import nd, gpu, gluon, autograd
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
from time import time

https://aws.amazon.com/ko/blogs/aws/sagemaker/

In [2]:
# -*- coding: utf-8 -*-
import mxnet as mx
import numpy as np
import pandas as pd
class Exchange(mx.gluon.data.Dataset):    
    def __init__(self,train=False, time_step=28, day=1, normalization_factor=100):        
        self.train=train        
        self.time_step=time_step        
        self.day=day        
        self.normalization_factor = normalization_factor        
        #self._data_preprocessing()    
        def __repr__(self):        
            return "'JPY_to_KRW'"    
        def __getitem__(self, idx):        
            return self._data[idx] , self._label[idx]    
        def __len__(self):        
            return len(self._data)    
        def _data_preprocessing(self):        
            #JPY data        
            data = pd.read_excel("datasets/jpy.xls")        # change the type of data to numpy        
            data = np.asarray(data)        
            data = data[:,-1:]        
            data = np.reshape(data,(-1,self.day))        
            data = data / self.normalization_factor        
            if self.train:            
                self._data = data[:len(data)-self.time_step].astype(np.float32)            
                self._label = data[1:len(data)-self.time_step+1].astype(np.float32)        
            else:            
                self._data = data[len(data)-self.time_step:].astype(np.float32)            
                self._label = np.zeros(np.shape(self._data)).astype(np.float32) # __getitem__' is required for proper operation. (It should be the same size as 'self._data' above.)


In [3]:
from mxnet.gluon.rnn.rnn_cell import LSTMCell

In [4]:
def BTC_to_USD(time_step,day,normalization_factor):
    dp = Exchange()
    training = gluon.data.DataLoader(Exchange(train=True,time_step=time_step, day=day,normalization_factor=normalization_factor), batch_size=time_step) #Loads data from a dataset and returns mini-batches of data.
    prediction = gluon.data.DataLoader(Exchange(train=False,time_step=time_step, day=day,normalization_factor=normalization_factor), batch_size=time_step)  # Loads data from a dataset and returns mini-batches of data.
    return training, prediction

In [5]:
def exchange_rate_model(epoch=1000, time_step=28, day=7, normalization_factor=100, save_period=1000 , load_period=1000 , learning_rate=0.001, ctx=mx.gpu(0)):

    ''' 28 time x 1 day '''
    #network parameter
    time_step = time_step # 28  step
    day = day # 1 day
    normalization_factor=normalization_factor
    num_hidden = 300
    training, test = BTC_to_USD(time_step,day,normalization_factor)
    path = "weights/LSTMCell_weights-{}.params".format(load_period)
    model = LSTMCell(num_hidden,day)
    model.hybridize()

    # weight initialization
    if os.path.exists(path):
        print("loading weights")
        model.load_params(filename=path ,ctx=ctx) # weights load
    else:
        print("initializing weights")
        model.collect_params().initialize(mx.init.Normal(sigma=0.01), ctx=ctx) # weights initialization

    trainer = gluon.Trainer(model.collect_params(), "adam", {"learning_rate": learning_rate})
    for i in tqdm(range(1,epoch+1,1)):
        for data, label in training:
            H_states=nd.zeros(shape=(1, num_hidden), ctx=ctx)
            C_states=nd.zeros(shape=(1, num_hidden), ctx=ctx)
            data = data.as_in_context(ctx)
            label = label.as_in_context(ctx)
            data = data.reshape(shape=(-1, time_step, day))
            data = nd.transpose(data=data, axes=(1, 0, 2))
            loss = 0

            with autograd.record():
                for j in range(time_step):
                    outputs , [H_states , C_states] = model(data[j],[H_states,C_states])
                    loss = loss + gluon.loss.L2Loss()(outputs, label[j].reshape(shape=outputs.shape))
            loss.backward()
            trainer.step(batch_size=1)
        cost = nd.mean(loss).asscalar()
        print(" epoch : {} , last batch cost : {}".format(i,cost))

        #weight_save
        if i % save_period==0:
            if not os.path.exists("weights"):
                os.makedirs("weights")

            print("saving weights")
            model.save_params("weights/LSTMCell_weights-{}.params".format(i))
    prediction(test, time_step, day, normalization_factor ,num_hidden, model, ctx)

In [6]:
def prediction(test_data, time_step, day, normalization_factor, num_hidden, model, ctx):

    for data, label in test_data:
        H_states = nd.zeros(shape=(1, num_hidden), ctx=ctx)
        C_states = nd.zeros(shape=(1, num_hidden), ctx=ctx)
        data = data.as_in_context(ctx)
        data = data.reshape(shape=(-1, time_step, day))
        data = nd.transpose(data=data, axes=(1, 0, 2))
        outputs_list=[]
        for j in range(time_step):
            outputs, [H_states, C_states] = model(data[j], [H_states, C_states])
            outputs_list.append(outputs.asnumpy())
    outputs_list = np.array(outputs_list) * normalization_factor
    outputs_list= np.reshape(outputs_list,(-1,))

    print("BTC-USD exchange rate prediction.")
    print("prediction value : {}".format(outputs_list[-1]))
    print("real value : {}".format(971.66))

In [7]:
exchange_rate_model(epoch=1000, time_step=28, day=7, normalization_factor=100, save_period=1000 , load_period=1000 , learning_rate=0.001, ctx=mx.gpu(0))

NotImplementedError: 